In [1]:
import pandas
import data_process
import torch
data  = pandas.read_csv('./motionClassify.csv')
l = len(data)
vocab = data_process.gen_vocab(data)
l



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Anaconda\envs\DL\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Anaconda\envs\DL\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\envs\DL\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Anaconda\envs\DL\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Anaconda\envs\DL\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\envs\DL\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found

50000

总共5w条数据，选取40000条用于训练，10000条用于测试


In [61]:
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)

In [62]:
Batch_size = 50
train_iter = torch.utils.data.DataLoader(data_train,Batch_size,shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

看看train_iter里面是什么

In [63]:
for X,y in train_iter:
    print(type(X),X.shape)
    print(X)
    print(y)
    print(len(y))
    break
    

<class 'torch.Tensor'> torch.Size([50, 600])
tensor([[    35, 170470,  30789,  ...,      0,      0,      0],
        [     3,    715,     44,  ...,      0,      0,      0],
        [     3,     44,   2175,  ...,      0,      0,      0],
        ...,
        [   618,   6212,    416,  ...,      0,      0,      0],
        [   131,   8481,    480,  ...,      0,      0,      0],
        [    31,   3043,   2909,  ...,      0,      0,      0]])
tensor([0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1])
50


In [64]:
torch.cuda.is_available()#有显卡的电脑会显示True

True

In [65]:
lr = 0.05
net = torch.nn.Linear(600,2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr)
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda:0')

In [66]:
from tqdm import tqdm
net.train()
net.to(device)
for i,(X,y) in tqdm(enumerate(train_iter),total=len(train_iter)):
    X = X.to(torch.float32)
    X = X.to(device)
    
    y = y.to(device)
    
    optimizer.zero_grad()
    y_hat = net(X)
    loss = criterion(y_hat,y)
    loss.backward()
    if (i+1)%100 == 0:
        print(f'batch{i+1},loss = {loss.item()}')
    optimizer.step()
    

 14%|█▎        | 109/800 [00:00<00:00, 1043.76it/s]

batch100,loss = 53092516.0


 27%|██▋       | 219/800 [00:00<00:00, 1048.77it/s]

batch200,loss = 23055002.0
batch300,loss = 47915012.0


 41%|████      | 325/800 [00:00<00:00, 1037.63it/s]

batch400,loss = 28336834.0

 68%|██████▊   | 548/800 [00:00<00:00, 1056.68it/s]


batch500,loss = 56291224.0
batch600,loss = 58388828.0


100%|██████████| 800/800 [00:00<00:00, 1047.49it/s]

batch700,loss = 30061118.0
batch800,loss = 40325424.0


In [67]:
for p in torch.nn.Linear(2,3).parameters():
    print(p)

Parameter containing:
tensor([[ 0.5071,  0.1108],
        [-0.3981,  0.5734],
        [-0.6195, -0.5232]], requires_grad=True)
Parameter containing:
tensor([-0.1774, -0.0401, -0.5667], requires_grad=True)


因为X中的元素在0到几千，net中的参数一般在0~1中，故对X先归一化试试看,在net中加一层softmax

In [68]:
def train(net,train_iter,device,optimizer,criterion):
    net.train()
    net.to(device)
    for i,(X,y) in tqdm(enumerate(train_iter),total=len(train_iter)):
        X = X.to(torch.float32)
        X = X.to(device)
        
        y = y.to(device)
        
        optimizer.zero_grad()
        y_hat = net(X)
        loss = criterion(y_hat,y)
        loss.backward()
        if (i+1)%100 == 0:
            tqdm.write(f'batch{i+1},loss = {loss.item()}')
        optimizer.step()

In [69]:
net = torch.nn.Sequential(
    torch.nn.Softmax(dim=1),
    torch.nn.Linear(600,2)
)

In [70]:
optimizer = torch.optim.SGD(net.parameters(),lr = 0.1)
train(net,train_iter,device,optimizer,criterion)

  9%|▉         | 73/800 [00:00<00:01, 710.04it/s]

batch100,loss = 0.6856198310852051


 31%|███       | 245/800 [00:00<00:00, 801.74it/s]

batch200,loss = 0.6942236423492432


 43%|████▎     | 343/800 [00:00<00:00, 858.60it/s]

...

batch600,loss = 0.6911211609840393


 93%|█████████▎| 742/800 [00:00<00:00, 940.44it/s]

batch700,loss = 0.690879762172699


100%|██████████| 800/800 [00:00<00:00, 894.68it/s]

batch800,loss = 0.6971619129180908


In [71]:
def test(net,test_iter,device):
    net.eval()
    net.to(device)
    with torch.no_grad():
        tp = 0
        cnt = 0
        for i,(X,y) in tqdm(enumerate(test_iter),total=len(test_iter)):
            X = X.to(torch.float32)
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            tp += (y==y_hat.argmax(dim=1)).sum()
            cnt += y.shape[0]
            
    print(f'accuracy = {tp/cnt}')

    

In [75]:
test(net,train_iter,device)
test(net,test_iter,device)

100%|██████████| 800/800 [00:00<00:00, 1450.70it/s]


accuracy = 0.5149999856948853


100%|██████████| 200/200 [00:00<00:00, 1544.83it/s]

accuracy = 0.499099999666214


全连接神经网络处理语言很差，在训练集上也不能收敛